In [ ]:
%pip install --upgrade --quiet pymupdf langchain langchain-core langchain-google-genai langchain-pinecone langchain-community sentence-transformers

In [ ]:
from langchain.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("/content/GPU.pdf")
documents = loader.load()


from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(documents)

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

# Use an open-source embedding model
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L12-v2",
    model_kwargs={'device': 'cpu'}
)

In [ ]:
# Create Pinecone vector store
from pinecone import Pinecone, ServerlessSpec
from google.colab import userdata

pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"))

In [ ]:
index_name = "multiagent-rag"

# Check if index already exists
if index_name not in [index['name'] for index in pc.list_indexes()]:
    # Create index if it doesn't exist
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
    print(f"Index '{index_name}' created.")
else:
    pinecone_index = pc.Index(index_name)
    print(f"Index '{index_name}' already exists and has been initialized")


In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=pinecone_index,embedding=embedding_model)

In [ ]:
vector_store.add_documents(chunks)

In [ ]:
class RetrievalAgent:
    def __init__(self):
        # Initialize embeddings
        self.embeddings = embedding_model

        # Connect to the existing vector store
        self.vector_store = vector_store

    def retrieve(self, query, k=3):
        """Retrieve the top-k most relevant documents for the query"""
        docs = self.vector_store.similarity_search_with_score(query, k=k)
        return docs

agent = RetrievalAgent()
results = agent.retrieve("How do Kernels work?")
for doc,score in results:
    print(f"Similarity={score:3f}")
    print(doc.page_content)
    print("-" * 80)

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

class ResponseGenerator:
    def __init__(self, retrieval_agent):
        self.retrieval_agent = retrieval_agent

        self.llm = ChatGoogleGenerativeAI(model="models/gemini-2.5-flash-preview-04-17",api_key=userdata.get("GOOGLE_API_KEY_1"))

        # Create prompt template
        self.prompt_template = PromptTemplate.from_template(
            """You are a helpful assistant that answers questions based on the provided context.

            Context:
            {context}

            Question: {query}

            Please provide a detailed and accurate answer based only on the information in the context.
            If the context doesn't contain relevant information to answer the question, say so.
            """
        )

        # Setup response chain
        self.chain = (
            {"context": self.retrieve_docs, "query": RunnablePassthrough()}
            | self.prompt_template
            | self.llm
            | StrOutputParser()
        )

    def retrieve_docs(self, query):
        docs = self.retrieval_agent.retrieve(query)
        return "\n\n".join([doc[0].page_content for doc in docs])

    def generate_response(self, query):
        return self.chain.invoke(query)


retrieval_agent = RetrievalAgent()
generator = ResponseGenerator(retrieval_agent)
response = generator.generate_response("What is a Kernel?")
print(response)